In [1]:
import configparser
from faker import Faker
import random
import pandas as pd
import os

#Путь к config
config_path = os.path.join(os.getcwd(), "config.ini")

# Чтение config
config = configparser.ConfigParser()
config.read(config_path ,encoding='utf-8')


# Инициализация Faker
fake = Faker()

# Получение данных из config
products = eval(config['Products']['PRODUCTS'])
categories = eval(config['Categories']['CATEGORIES'])
shop_qty=eval(config['Array_Dimension']['SHOP_QTY'])
num_records=eval(config['Array_Dimension']['NUM_RECORDS'])
data_folder = config['Paths']['DATA_FOLDER']
discounts=eval(config['Discounts']['DISCOUNTS'])
# Генерация данных по продажам

def generate_sales_data(shop_qty,num_records):
    sales_data = []
    for _ in range (num_records):
     for shop in range(1, shop_qty + 1):
         row = {
            
            "shop_num": shop,
            "cash_num": random.randint(1,3),
            "cheque_id": fake.numerify(text="cheque_№-##########"),
            "date":  fake.date_time_this_year(before_now=True),
            "product": random.choice(products),
            "category": random.choice(categories),
            "quantity": random.randint(1, 10),
            "price": random.randint(100, 5000),
            "discount_perc":random.choice(discounts)
        }
         sales_data.append(row)
    
    return  pd.DataFrame(sales_data)


# Создание файлов для загрузки 
def files_making(df):
    shops = df['shop_num'].unique().tolist()
    cash_num = df['cash_num'].unique().tolist()

    if not os.path.exists(data_folder):
        os.makedirs(data_folder)

    for shop in shops:
      for cash in cash_num:
          file_df = df[(df['shop_num'] == shop) & (df['cash_num']== cash)]
          file_path = os.path.join(data_folder, f'{shop}_{cash}.csv')
          file_df.to_csv(file_path, index=False)


          


# Выполнение 
def main():
   try:
      fake_sales = generate_sales_data(shop_qty, num_records)
      files_making(fake_sales)
   except Exception as e:
      print(f"Ошибка: {e}")
   

if __name__ == "__main__":
   main()